# Visualise the Tracked Marine Heatwaves

In [ ]:
import xarray as xr
import numpy as np
import dask
from getpass import getuser
from pathlib import Path

import spot_the_blOb.helper as hpc
from spot_the_blOb.plotter import *

from matplotlib.colors import ListedColormap

In [ ]:
# Start Dask Cluster
client = hpc.StartLocalCluster(n_workers=32, n_threads=4)

In [ ]:
# Import Tracked MHW DataSet

file_name = Path('/scratch') / getuser()[0] / getuser() / 'mhws' / 'MHWs_tracked.nc'
blobs_ds = xr.open_dataset(file_name, chunks={'time': 50, 'lat': -1, 'lon': -1})
blobs_ds

In [ ]:
# Extract the MHW ID Field

blobs = blobs_ds.ID_field
blobs

## Plot some Blobby Shapes

In [ ]:
cm = ListedColormap(np.random.random(size=(int(1e5), 3)).tolist())

blobs_day1 = blobs.sel(time=slice('2020-01-01', '2020-12-31')).resample(time='MS').first()
blobs_day1.pplot_col(col_wrap=3, var_units='MHW ID', cmap=cm, cperc=[0, 100])

## Global Blob Frequency

In [ ]:
event_frequency = ((blobs != 0) & ~np.isnan(blobs)).astype(float).mean('time')
event_frequency.pplot(var_units='MHW Frequency', cmap='hot_r', cperc=[0,96])

## Find & Plot a Few of the Longest Events

In [ ]:
events_duration = blobs_ds.time_end - blobs_ds.time_start
longest_events = events_duration.sortby(events_duration, ascending=False).ID

for ID in longest_events[:10].values:
    print(f"ID: {ID}, Start Day: {blobs_ds.time_start.sel(ID=ID)}, Duration: {events_duration.sel(ID=ID).item()} days")

In [ ]:
long_blobs = blobs.sel(ID=longest_events[:9])
long_blobs_local_duration = ((long_blobs != 0) & ~np.isnan(long_blobs)).astype(float).sum('time')

long_blobs_local_duration.pplot_col(col_wrap=3, var_units='Duration (days)', cmap='hot_r', cperc=[0, 100])

## Time-Series Plot of Events Area

In [ ]:
events_areas = blobs_ds.area * 1.0e5/16.0 # km^2 (very rough estimate, since "area" here is in units of 0.25x0.25 degree grid cells...)

plt.figure(figsize=(14, 8))

events_areas.plot.line(x='time', hue='ID', alpha=0.5, add_legend=False)
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Rough Event Area [km^2]')
plt.show()
